In [1]:
# import package
import pandas as pd 
import random
import numpy as np 
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import skew
from seaborn import regplot
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.cluster import KMeans
import statsmodels.api as sm
from scipy.stats import gamma,expon,norm,chi2

In [2]:
# loading data
Batting     = pd.read_csv(r'C:\Users\ASUS\Desktop\baseballdatabank-master\baseballdatabank-master\core\Batting.csv');
People      = pd.read_csv(r'C:\Users\ASUS\Desktop\baseballdatabank-master\baseballdatabank-master\core\People.csv');
Salaries    = pd.read_csv(r'C:\Users\ASUS\Desktop\baseballdatabank-master\baseballdatabank-master\core\Salaries.csv');
Fielding    = pd.read_csv(r'C:\Users\ASUS\Desktop\baseballdatabank-master\baseballdatabank-master\core\Fielding.csv');
Appearances = pd.read_csv(r'C:\Users\ASUS\Desktop\baseballdatabank-master\baseballdatabank-master\core\Appearances.csv');

In [3]:
field = Fielding.fillna(0)

In [4]:
field

,playerID,yearID,stint,teamID,lgID,POS,G,GS,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR
0,abercda01,1871,1,TRO,0,SS,1,1.0,24.0,1,3,2.0,0,0.0,0.0,0.0,0.0,0.0
1,addybo01,1871,1,RC1,0,2B,22,22.0,606.0,67,72,42.0,5,0.0,0.0,0.0,0.0,0.0
2,addybo01,1871,1,RC1,0,SS,3,3.0,96.0,8,14,7.0,0,0.0,0.0,0.0,0.0,0.0
3,allisar01,1871,1,CL1,0,2B,2,0.0,18.0,1,4,0.0,0,0.0,0.0,0.0,0.0,0.0
4,allisar01,1871,1,CL1,0,OF,29,29.0,729.0,51,3,7.0,1,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144763,zimmebr02,2020,1,BAL,AL,P,2,1.0,21.0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0
144764,zimmejo02,2020,1,DET,AL,P,3,2.0,17.0,0,1,0.0,0,0.0,0.0,0.0,0.0,0.0
144765,zimmeky01,2020,1,KCA,AL,P,16,1.0,69.0,2,4,0.0,1,0.0,0.0,0.0,0.0,0.0
144766,zuberty01,2020,1,KCA,AL,P,23,0.0,66.0,0,2,0.0,0,0.0,0.0,0.0,0.0,0.0


In [5]:
# define search function
def search(data,y_i=None,y_f=None,teamID=None,playerID=None):  
    if type(y_i)!=int:
        data1=data
    else:data1 = data[data['yearID']>=y_i];
    if type(y_f)!=int:
        data1=data1
    else:data1= data1[data1['yearID']<=y_f];
    if type(playerID)!=str:
        data2=data1
    else:data2 = data1[data1['playerID']==playerID];
    if type(teamID)!=str:
        data3=data2
    else:data3 = data2[data2['teamID']==teamID];
    return data3

In [6]:
mbb = pd.read_csv('mbb_1990.csv').drop('Unnamed: 0',axis=1)
mbb = mbb[mbb['yearID']<=2015]
mbb.head()

,playerID,yearID,seniority,teamID,lgID,G,AB,R,H,2B,...,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,age
0,bucknbi01,1990,22,BOS,AL,22,43,4,8,0,...,0,0,3,2,2,0,1,1,1,41
1,dempsri01,1990,22,LAN,NL,62,128,13,25,5,...,1,0,23,29,0,0,0,0,8,41
2,dempsri01,1991,23,ML4,AL,61,147,15,34,5,...,0,2,23,20,1,0,1,3,7,42
3,dempsri01,1992,24,BAL,AL,8,9,2,1,0,...,0,0,2,1,0,0,0,0,1,43
4,fiskca01,1990,21,CHA,AL,137,452,65,129,21,...,7,2,61,73,8,7,0,1,12,43


In [7]:
# merge batting record which is splitted 
w = field.merge(mbb['playerID']).drop_duplicates(subset=['playerID','yearID'])
merge_field = pd.DataFrame()
for i,j in zip(w['yearID'],w['playerID']):
    f = search(field,y_i=i,y_f=i,playerID=j)
    if len(f)!=1:
        d = pd.DataFrame(f.sum()).T
        d.loc[:,'playerID']=f['playerID'].drop_duplicates().values
        d.loc[:,'yearID']=f['yearID'].drop_duplicates().values
        d.loc[:,'teamID']=f['teamID'].values[0]
        d.loc[:,'lgID']=f['lgID'].values[0]
        d.loc[:,'POS']=f['POS'].values[0]
        f = d
    merge_field = pd.concat([merge_field,f])
merge_field = merge_field.astype({'G':int, 'GS':int,'InnOuts':int,
                                            'PO':int, 'A':int, 'E':int, 'DP':int, 
                                            'PB':int, 'WP':int, 'SB':int, 'CS':int, 'ZR':int})

In [8]:
merge_field.drop_duplicates(subset=['playerID','yearID'])

,playerID,yearID,stint,teamID,lgID,POS,G,GS,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR
64895,dempsri01,1969,1,MIN,AL,C,3,0,32,5,0,1,0,0,0,1,0,0
66122,dempsri01,1970,1,MIN,AL,C,3,1,48,12,0,1,0,0,0,2,0,0
67372,dempsri01,1971,1,MIN,AL,C,6,4,111,30,4,2,0,0,0,4,2,0
68559,dempsri01,1972,1,MIN,AL,C,23,8,339,67,5,1,0,2,0,5,4,0
69748,dempsri01,1973,1,NYA,AL,C,5,3,93,9,0,2,0,0,0,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,sanchga02,2017,2,NYA,AL,1B,106,99,2652,942,60,13,5,16,0,37,23,0
140509,sanchga02,2018,1,NYA,AL,C,76,74,1959,731,35,6,5,18,0,28,12,0
142616,sanchga02,2019,1,NYA,AL,C,90,86,2228,793,32,15,5,7,0,36,11,0
144420,sanchga02,2020,1,NYA,AL,C,41,38,965,347,14,6,2,5,0,13,5,0


In [9]:
merge_field = merge_field.rename(columns={'SB':'SB_c','CS':'CS_c'})

In [10]:
merge_field

,playerID,yearID,stint,teamID,lgID,POS,G,GS,InnOuts,PO,A,E,DP,PB,WP,SB_c,CS_c,ZR
64895,dempsri01,1969,1,MIN,AL,C,3,0,32,5,0,1,0,0,0,1,0,0
66122,dempsri01,1970,1,MIN,AL,C,3,1,48,12,0,1,0,0,0,2,0,0
67372,dempsri01,1971,1,MIN,AL,C,6,4,111,30,4,2,0,0,0,4,2,0
68559,dempsri01,1972,1,MIN,AL,C,23,8,339,67,5,1,0,2,0,5,4,0
69748,dempsri01,1973,1,NYA,AL,C,5,3,93,9,0,2,0,0,0,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,sanchga02,2017,2,NYA,AL,1B,106,99,2652,942,60,13,5,16,0,37,23,0
140509,sanchga02,2018,1,NYA,AL,C,76,74,1959,731,35,6,5,18,0,28,12,0
142616,sanchga02,2019,1,NYA,AL,C,90,86,2228,793,32,15,5,7,0,36,11,0
144420,sanchga02,2020,1,NYA,AL,C,41,38,965,347,14,6,2,5,0,13,5,0


### output csv